In [1]:
import os
from typing import Optional, Tuple

import gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer, PrioritizedVectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger


from Custom_Classes import CustomNet
from Custom_Classes import CustomCollector
from Custom_Classes import CustomParallelToAECWrapper

#from CustomClass_multi_head import CustomNet
from Custom_Classes_simplified import CustomNetSimple
#from Custom_Classes_simplified import CustomCollectorSimple
#from Custom_Classes_simplified import CustomParallelToAECWrapperSimple

from CustomClasses_Transformer_Reduced import CustomNetReduced
import importlib

from DroneEnv import MultiDroneEnv
from tianshou_DQN import train


model = "CustomNetSimple" # "CustomNet" or "CustomNetSimple" or "CustomNetReduced"
test_num = "Eval_TBTA_01_max30agents_timeRew"

train_env_num = 10
test_env_num = 10

name = model + test_num

load_policy_name = f'policy_CustomNetSimple1605_01_1_Priorized_1605_01_1_Priorized.pth'
save_policy_name = f'policy_{name}.pth'
policy_path = "dqn_Custom"
load_model = False

log_path = os.path.join('./', "Logs", "dqn", name)

dqn_params = {"discount_factor": 0.99, 
              "estimation_step": 1, 
              "target_update_freq": 100,
              "optminizer": "Adam",
              "lr": 1e-4  }

trainer_params = {"max_epoch": 200,
                  "step_per_epoch": 600 * train_env_num,
                  "step_per_collect": 50 * train_env_num,
                  "episode_per_test": 10 * test_env_num,
                  "batch_size" : 32,
                  "update_per_step": 0.1,
                  "tn_eps_max": 0.8,
                  "ts_eps_max": 0.001,
                  }

Run_Data = f'{name}\n\
        Loaded_Model: {load_policy_name if load_model == True else "no"} \n\
        log_path: {log_path} \n\
        train/test_env_num: {train_env_num} / {test_env_num} \n\
        model: {model} \n\
        dqn_params: {dqn_params} \n\
        trainer_params: {trainer_params} \n\
        obs: Task Info -> Dist | Quality for each drone \
            agents= F1:2, R1:4 | tasks= Rec:15, Att:5'

model_load_path = os.path.join(policy_path, load_policy_name)  
model_save_path = os.path.join(policy_path, save_policy_name)        
os.makedirs(os.path.join(policy_path), exist_ok=True)  
os.makedirs(os.path.join(log_path), exist_ok=True)


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    policy_load_path = None
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    
    env = _get_env()
    agent_name = env.agents[0]  # Get the name of the first agent
           
    agent_observation_space = env.observation_space('agent0') # assuming 'agent0' is a valid agent name
    state_shape_agent_position = agent_observation_space["agent_position"].shape[0]
    state_shape_agent_state = agent_observation_space["agent_state"].shape[0]
    state_shape_agent_type = agent_observation_space["agent_type"].shape[0]
    state_shape_next_free_time = agent_observation_space["next_free_time"].shape[0]
    state_shape_position_after_last_task = agent_observation_space["position_after_last_task"].shape[0]       
    #state_shape_agent_relay_area = agent_observation_space["agent_relay_area"].shape[0]
        
    state_shape_agent = (state_shape_agent_position + state_shape_agent_state +
                     state_shape_agent_type+ state_shape_next_free_time + state_shape_position_after_last_task #+                     
                     #state_shape_agent_relay_area
                     )                 
    

    state_shape_task = env.observation_space('agent0')["tasks_info"].shape[0]
                  
    action_shape = env.action_space[agent_name].shape[0]
    #action_shape = env.action_space[agent_name].n
               
    if agent_learn is None:
        # model
        if model == "CustomNet":        
            net = CustomNet(
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")
        
        if model == "CustomNetSimple":
            net = CustomNetSimple(            
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")
        
        if model == "CustomNetReduced":
            net = CustomNetReduced(            
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")

    
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=dqn_params["lr"])
    
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor= dqn_params["discount_factor"],
            estimation_step=dqn_params["estimation_step"],
            target_update_freq=dqn_params["target_update_freq"],
        )  
        
        if load_model == True:
            # Load the saved checkpoint             
            agent_learn.load_state_dict(torch.load(model_load_path))
            print(f'Loaded-> {model_load_path}')
            
        
        agents = [agent_learn for _ in range(len(env.agents))]
        
    policy = MultiAgentPolicyManager(agents, env)    
        
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    env_paralell = MultiDroneEnv()
    #env = parallel_to_aec_wrapper(env_paralell)    
    env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)

print(Run_Data)

w:\OneDrive\Doutorado\SwarmSimASA\PettingZoo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CustomNetSimpleEval_TBTA_01_max30agents_timeRew
        Loaded_Model: no 
        log_path: ./Logs\dqn\CustomNetSimpleEval_TBTA_01_max30agents_timeRew 
        train/test_env_num: 10 / 10 
        model: CustomNetSimple 
        dqn_params: {'discount_factor': 0.99, 'estimation_step': 1, 'target_update_freq': 100, 'optminizer': 'Adam', 'lr': 0.0001} 
        trainer_params: {'max_epoch': 200, 'step_per_epoch': 6000, 'step_per_collect': 500, 'episode_per_test': 100, 'batch_size': 32, 'update_per_step': 0.1, 'tn_eps_max': 0.8, 'ts_eps_max': 0.001} 
        obs: Task Info -> Dist | Quality for each drone             agents= F1:2, R1:4 | tasks= Rec:15, Att:5


In [2]:
if __name__ == "__main__":
                        
    torch.set_grad_enabled(True) 
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(train_env_num)])
    test_envs = DummyVectorEnv([_get_env for _ in range(test_env_num)]) 

    # seed
    seed = 1
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()
    

    # ======== Step 3: Collector setup =========
    train_collector = CustomCollector(
        policy,
        train_envs,
        #VectorReplayBuffer(100_000, len(train_envs)),
        PrioritizedVectorReplayBuffer( 100_000, len(train_envs), alpha=0.6, beta=0.4) ,       
        exploration_noise=True        
    )
    test_collector = CustomCollector(policy, test_envs, exploration_noise=True)
     
    train_collector.collect(n_step=trainer_params['batch_size'] * train_env_num)
    #test_collector.collect(n_step=trainer_params['batch size'] * train_env_num)
    
    # ======== tensorboard logging setup =========
    #         
    writer = SummaryWriter(log_path)
    writer.add_text(name, str(Run_Data))
    logger = TensorboardLogger(writer)
        
    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):                
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 9939.0

    def train_fn(epoch, env_step):
        epsilon = trainer_params['tn_eps_max'] - (trainer_params['tn_eps_max'] - trainer_params['tn_eps_max']/100)*(epoch/trainer_params['max_epoch'])  
        policy.policies[agents[0]].set_eps(epsilon)

    def test_fn(epoch, env_step):
        epsilon = trainer_params['ts_eps_max']#0.01#max(0.001, 0.1 - epoch * 0.001)
        policy.policies[agents[0]].set_eps(epsilon)
        
    def reward_metric(rews):       
        #print(rews)
        return rews.mean()#[:,0]
                           
    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,        
        max_epoch=trainer_params['max_epoch'],
        step_per_epoch=trainer_params['step_per_epoch'],
        step_per_collect=trainer_params['step_per_collect'],
        episode_per_test= trainer_params['episode_per_test'],
        batch_size=trainer_params['batch_size'],
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=trainer_params['update_per_step'],
        logger=logger,
        test_in_train=False,
        reward_metric=reward_metric,
        show_progress = True
        )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[0]])")

w:\OneDrive\Doutorado\SwarmSimASA\PettingZoo\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


.


Epoch #1: 6001it [00:57, 104.27it/s, agent0/loss=44.005, agent1/loss=48.470, agent2/loss=54.151, agent3/loss=52.552, agent4/loss=51.525, agent5/loss=51.641, agent6/loss=53.283, agent7/loss=53.875, env_step=6000, len=20, n/ep=20, n/st=500, rew=-61.19]                          


Epoch #1: test_reward: -122.731227 ± 0.000000, best_reward: -122.731227 ± 0.000000 in #1


Epoch #2: 6001it [00:56, 106.81it/s, agent0/loss=45.230, agent1/loss=46.569, agent2/loss=52.473, agent3/loss=53.451, agent4/loss=52.020, agent5/loss=50.733, agent6/loss=46.244, agent7/loss=48.068, env_step=12000, len=20, n/ep=20, n/st=500, rew=-65.33]                          


...

Epoch #56: test_reward: -102.312722 ± 0.000000, best_reward: -67.762214 ± 0.000000 in #18


Epoch #57: 6001it [01:22, 72.44it/s, agent0/loss=41.060, agent1/loss=43.335, agent2/loss=48.592, agent3/loss=52.385, agent4/loss=47.117, agent5/loss=48.352, agent6/loss=46.800, agent7/loss=47.413, env_step=342000, len=20, n/ep=20, n/st=500, rew=-61.42]                           


Epoch #57: test_reward: -102.196611 ± 0.000000, best_reward: -67.762214 ± 0.000000 in #18


Epoch #58:  25%|##5       | 1500/6000 [00:20<01:00, 74.55it/s, agent0/loss=42.089, agent1/loss=41.102, agent2/loss=47.246, agent3/loss=48.743, agent4/loss=45.764, agent5/loss=50.565, agent6/loss=42.312, agent7/loss=50.357, env_step=343000, len=20, n/ep=20, n/st=500, rew=-62.76]


KeyboardInterrupt: 

In [ ]:

from typing import Optional, Tuple

import numpy as np
import torch
from tianshou.env import DummyVectorEnv
from tianshou.trainer import offpolicy_trainer
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
import torch
#from Custom_Classes import CustomCollector

# Create a new instance of the policy with the same architecture as the saved policy
policy, optim, _ = _get_agents()
model_save_path = os.path.join("dqn_Custom", save_policy_name)        

# Load the saved checkpoint
policy_test = policy.policies['agent0']
policy_test.load_state_dict(torch.load(model_save_path ))

envs = DummyVectorEnv([_get_env for _ in range(1)])

envs.max_time_steps = 200
#policy_test.policies['agent0'].eval()
#policy.policies['agent0'].set_eps(0.9)

policy_test.eval()
policy_test.set_eps(0.00)


#collector = CustomCollector(policy.policies['agent0'], envs, exploration_noise=True)
#collector = CustomCollector(policy_test, envs, exploration_noise=False)
collector = CustomCollector(policy, envs, exploration_noise=True)

results = collector.collect(n_episode=10)
#collector.collect(n_episode=1, render=1 / 5000)




w:\OneDrive\Doutorado\SwarmSimASA\PettingZoo\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [ ]:
results

{'n/ep': 10,
 'n/st': 400,
 'rews': array([[146.60998543, 146.60998543, 146.60998543, 146.60998543,
         146.60998543, 146.60998543, 146.60998543, 146.60998543],
        [149.65674368, 149.65674368, 149.65674368, 149.65674368,
         149.65674368, 149.65674368, 149.65674368, 149.65674368],
        [150.12206002, 150.12206002, 150.12206002, 150.12206002,
         150.12206002, 150.12206002, 150.12206002, 150.12206002],
        [150.34100261, 150.34100261, 150.34100261, 150.34100261,
         150.34100261, 150.34100261, 150.34100261, 150.34100261],
        [136.37620211, 136.37620211, 136.37620211, 136.37620211,
         136.37620211, 136.37620211, 136.37620211, 136.37620211],
        [145.54718066, 145.54718066, 145.54718066, 145.54718066,
         145.54718066, 145.54718066, 145.54718066, 145.54718066],
        [144.85799617, 144.85799617, 144.85799617, 144.85799617,
         144.85799617, 144.85799617, 144.85799617, 144.85799617],
        [146.66782528, 146.66782528, 146.6678252

In [ ]:
max(results['rews'])
print(np.mean(results['rews'][results['rews'] > -10]))


#create a function  to print a histogram of the results['rews']
import matplotlib.pyplot as plt
plt.hist(results['rews'][results['rews'] > -10], bins=100)
plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
from turtle import st
import torch
from tianshou.data import Batch

# load policy as in your original code
policy, optim, _ = _get_agents()
model_save_path = os.path.join("dqn_Custom", save_policy_name)        
policy_test = policy.policies['agent0']
state_saved = torch.load(model_save_path )
print(policy_test)
policy_test.load_state_dict(state_saved)
policy_test.eval()
policy_test.set_eps(0.00)

# initialize your environment
#env = DummyVectorEnv([_get_env for _ in range(1)])
env = MultiDroneEnv(None)
env.max_time_steps = 100

# simulate the interaction with the environment manually
for episode in range(1):  # simulate 10 episodes
    
    obs, _  = env.reset(seed=episode)         
    info         = env.get_initial_state()
    
    drones = info["drones"]
    tasks = info["tasks"]
        
    done = {0 : False}
    truncations = {0 : False}
    
    episodo_reward = 0
    #obs, reward, done, truncations, info = env.step(action)

    while not all(done.values()) and not all(truncations.values()):
        
        agent_id = "agent" + str(env.agent_selector._current_agent)
        # Create a Batch of observations
        obs_batch = Batch(obs=obs[agent_id], info=[{}])  # add empty info for each observation
        
        #print(obs_batch)
        # Forward the batch of observations through the policy to get the actions
        action = policy_test(obs_batch).act
        action = {agent_id : action[0]}
       
        obs, reward, done, truncations, info = env.step(action)
          
        episodo_reward += sum(reward.values())/env.n_agents

       

print(episodo_reward)
